# Quantum Encoding Methods for Earth Observation Data

## Introduction to Quantum Machine Learning for Remote Sensing

In this notebook, we'll explore different methods of encoding classical Earth observation (EO) data into quantum states. This is a crucial first step in quantum machine learning, as quantum computers operate on quantum states rather than classical bits.

**Why Quantum Encoding Matters for Earth Observation:**
- EO datasets are often high-dimensional (multispectral/hyperspectral imagery)
- Quantum states can represent exponentially large feature spaces
- Different encoding strategies affect which features are emphasized
- Choice of encoding impacts the performance of quantum machine learning models

We'll use the **EuroSAT dataset**, which contains Sentinel-2 satellite images classified into 10 land use categories.

## Setup and Dependencies

First, let's install and import the necessary libraries:

In [ ]:
# Install required packages
!pip install pennylane scikit-learn -q

In [ ]:
import pennylane as qml
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os
import zipfile
import urllib.request
from pathlib import Path

# Set random seed for reproducibility
np.random.seed(42)

print(f"PennyLane version: {qml.__version__}")
print(f"NumPy version: {np.__version__}")

## Download and Prepare EuroSAT Dataset

The EuroSAT dataset contains RGB images (64×64 pixels) from Sentinel-2 satellites, covering 10 land use classes:
- Annual Crop, Forest, Herbaceous Vegetation, Highway, Industrial, Pasture, Permanent Crop, Residential, River, Sea/Lake

In [ ]:
def download_and_extract_eurosat(data_dir="./eurosat_data"):
    """Download and extract the EuroSAT dataset"""
    
    url = "https://madm.dfki.de/files/sentinel/EuroSAT.zip"
    zip_path = os.path.join(data_dir, "EuroSAT.zip")
    
    # Create directory if it doesn't exist
    os.makedirs(data_dir, exist_ok=True)
    
    # Download if not already present
    if not os.path.exists(zip_path):
        print(f"Downloading EuroSAT dataset from {url}...")
        urllib.request.urlretrieve(url, zip_path)
        print("Download complete!")
    else:
        print("Dataset already downloaded.")
    
    # Extract if not already extracted
    extract_path = os.path.join(data_dir, "2750")
    if not os.path.exists(extract_path):
        print("Extracting dataset...")
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(data_dir)
        print("Extraction complete!")
    else:
        print("Dataset already extracted.")
    
    return extract_path

# Download and extract the dataset
dataset_path = download_and_extract_eurosat()

## Load and Visualize Sample EO Data

In [ ]:
def load_sample_images(dataset_path, n_samples=5):
    """Load sample images from different classes"""
    
    classes = sorted([d for d in os.listdir(dataset_path) if os.path.isdir(os.path.join(dataset_path, d))])
    samples = []
    
    for cls in classes[:n_samples]:
        class_path = os.path.join(dataset_path, cls)
        images = [f for f in os.listdir(class_path) if f.endswith('.jpg')]
        if images:
            img_path = os.path.join(class_path, images[0])
            img = Image.open(img_path)
            samples.append((cls, np.array(img)))
    
    return samples

# Load samples
samples = load_sample_images(dataset_path)

# Visualize
fig, axes = plt.subplots(1, len(samples), figsize=(15, 3))
for idx, (class_name, img) in enumerate(samples):
    axes[idx].imshow(img)
    axes[idx].set_title(class_name)
    axes[idx].axis('off')
plt.tight_layout()
plt.show()

## Preprocessing: From Satellite Image to Feature Vector

Before we can encode data into quantum states, we need to preprocess our images into normalized feature vectors.

In [ ]:
def preprocess_image(img, target_size=8):
    """
    Preprocess satellite image for quantum encoding
    
    Args:
        img: Input image (H x W x C)
        target_size: Number of features to extract
    
    Returns:
        Normalized feature vector
    """
    # Convert to grayscale if RGB
    if len(img.shape) == 3:
        img_gray = np.mean(img, axis=2)
    else:
        img_gray = img
    
    # Resize to small patch and flatten
    from PIL import Image as PILImage
    img_resized = PILImage.fromarray(img_gray.astype(np.uint8)).resize((int(np.sqrt(target_size)), 
                                                                          int(np.sqrt(target_size))))
    features = np.array(img_resized).flatten()
    
    # Normalize to [0, 1]
    features = features / 255.0
    
    # Further normalize to have unit norm for some encoding schemes
    features = features / (np.linalg.norm(features) + 1e-10)
    
    return features

# Example: preprocess one image
sample_img = samples[0][1]
features = preprocess_image(sample_img, target_size=8)

print(f"Original image shape: {sample_img.shape}")
print(f"Feature vector shape: {features.shape}")
print(f"Feature vector (first 8 values): {features}")
print(f"Feature vector norm: {np.linalg.norm(features):.4f}")

# Part 1: Basis Encoding (Digital Encoding)

## Theory

**Basis encoding** (also called digital encoding) is the most straightforward encoding method. It directly maps classical binary data to the computational basis states of qubits.

### Mathematical Formulation

For a classical bit string $x = x_1 x_2 ... x_n$ where $x_i \in \{0, 1\}$, basis encoding creates:

$$|x\rangle = |x_1\rangle \otimes |x_2\rangle \otimes ... \otimes |x_n\rangle$$

### Properties:
- **Direct mapping**: Each classical bit → one qubit
- **No superposition**: States are computational basis states
- **Information capacity**: $n$ qubits encode $n$ classical bits
- **Use case**: Good for discrete/categorical features

### For Earth Observation:
In EO applications, we might use basis encoding for:
- Land cover classification labels (binary or one-hot encoded)
- Thresholded pixel values (cloud/no-cloud, water/land)
- Discretized spectral indices (NDVI ranges)

In [ ]:
def basis_encoding(features, threshold=0.5):
    """
    Encode features using basis encoding
    
    Args:
        features: Normalized feature vector [0, 1]
        threshold: Threshold for binarization
    
    Returns:
        Binary string representation
    """
    # Binarize features
    binary_features = (features > threshold).astype(int)
    return binary_features

# Apply basis encoding
n_qubits = 8
binary_encoded = basis_encoding(features[:n_qubits])

print("Basis Encoding Example:")
print(f"Original features: {features[:n_qubits]}")
print(f"Binary encoded:    {binary_encoded}")
print(f"Quantum state:     |{''.join(map(str, binary_encoded))}⟩")

### Implementing Basis Encoding in PennyLane

In [ ]:
# Define quantum device
dev_basis = qml.device('default.qubit', wires=n_qubits)

@qml.qnode(dev_basis)
def basis_encoding_circuit(binary_data):
    """
    Create a quantum circuit with basis encoding
    
    Args:
        binary_data: Binary feature vector
    """
    # Apply X gates where bit is 1
    for i, bit in enumerate(binary_data):
        if bit == 1:
            qml.PauliX(wires=i)
    
    # Return state vector
    return qml.state()

# Execute circuit
quantum_state_basis = basis_encoding_circuit(binary_encoded)

# Visualize
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 4))

# Classical data
ax1.bar(range(n_qubits), features[:n_qubits], color='blue', alpha=0.6, label='Original')
ax1.bar(range(n_qubits), binary_encoded, color='red', alpha=0.6, label='Binarized')
ax1.set_xlabel('Feature Index')
ax1.set_ylabel('Value')
ax1.set_title('Classical Feature → Binary Encoding')
ax1.legend()
ax1.set_xticks(range(n_qubits))

# Quantum state amplitudes
basis_states = [f"|{i:0{n_qubits}b}⟩" for i in range(2**n_qubits)]
amplitudes = np.abs(quantum_state_basis)**2

# Only show non-zero amplitude
non_zero_idx = np.where(amplitudes > 1e-10)[0]
ax2.bar(non_zero_idx, amplitudes[non_zero_idx], color='green')
ax2.set_xlabel('Basis State')
ax2.set_ylabel('Probability')
ax2.set_title(f'Quantum State: {basis_states[non_zero_idx[0]]}')
ax2.set_xticks(non_zero_idx)
ax2.set_xticklabels([basis_states[i] for i in non_zero_idx], rotation=45)

plt.tight_layout()
plt.show()

print(f"\nThe encoded state is a single basis state: {basis_states[non_zero_idx[0]]}")
print(f"Probability of measuring this state: {amplitudes[non_zero_idx[0]]:.4f}")

# Part 2: Amplitude Encoding (Dense Encoding)

## Theory

**Amplitude encoding** embeds classical data into the amplitudes of a quantum state, providing an exponential advantage in data representation.

### Mathematical Formulation

For a classical vector $\mathbf{x} = (x_1, x_2, ..., x_N)$ where $N = 2^n$, amplitude encoding creates:

$$|\psi\rangle = \sum_{i=1}^{N} x_i |i\rangle$$

where the state must be normalized: $\sum_{i=1}^{N} |x_i|^2 = 1$

### Properties:
- **Exponential capacity**: $n$ qubits encode $2^n$ classical values
- **Superposition**: Data encoded in quantum amplitudes
- **Normalization required**: Features must form valid quantum state
- **Use case**: Optimal for high-dimensional continuous data

### For Earth Observation:
Perfect for encoding:
- Normalized pixel intensities across spectral bands
- Spatial feature vectors from image patches
- Compressed representations of hyperspectral signatures

**Challenge**: Requires $2^n$ amplitudes, so we need $\text{log}_2(\text{features})$ qubits

In [ ]:
def amplitude_encoding_prep(features):
    """
    Prepare features for amplitude encoding
    
    Args:
        features: Feature vector
    
    Returns:
        Normalized feature vector padded to power of 2
    """
    # Pad to nearest power of 2
    n_features = len(features)
    n_qubits = int(np.ceil(np.log2(n_features)))
    n_amplitudes = 2 ** n_qubits
    
    # Pad with zeros
    padded = np.zeros(n_amplitudes)
    padded[:n_features] = features
    
    # Normalize
    norm = np.linalg.norm(padded)
    if norm > 0:
        padded = padded / norm
    
    return padded, n_qubits

# Prepare data
amplitudes, n_qubits_amp = amplitude_encoding_prep(features)

print(f"Original features: {len(features)} values")
print(f"Required qubits: {n_qubits_amp}")
print(f"Amplitude vector: {len(amplitudes)} values (2^{n_qubits_amp})")
print(f"First 8 amplitudes: {amplitudes[:8]}")
print(f"Norm check: {np.linalg.norm(amplitudes):.6f} (should be 1.0)")

### Implementing Amplitude Encoding in PennyLane

In [ ]:
dev_amplitude = qml.device('default.qubit', wires=n_qubits_amp)

@qml.qnode(dev_amplitude)
def amplitude_encoding_circuit(amplitudes):
    """
    Create quantum circuit with amplitude encoding
    
    Args:
        amplitudes: Normalized amplitude vector
    """
    # PennyLane's built-in amplitude embedding
    qml.AmplitudeEmbedding(features=amplitudes, wires=range(n_qubits_amp), normalize=True)
    
    return qml.state()

# Execute circuit
quantum_state_amplitude = amplitude_encoding_circuit(amplitudes)

# Visualize
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 4))

# Classical amplitudes
ax1.bar(range(len(amplitudes)), amplitudes, color='blue', alpha=0.7)
ax1.set_xlabel('Index')
ax1.set_ylabel('Amplitude')
ax1.set_title('Classical Data (Normalized)')
ax1.axhline(y=0, color='k', linestyle='-', linewidth=0.5)

# Quantum state amplitudes
state_amplitudes = np.abs(quantum_state_amplitude)
ax2.bar(range(len(state_amplitudes)), state_amplitudes, color='green', alpha=0.7)
ax2.set_xlabel('Basis State Index')
ax2.set_ylabel('|Amplitude|')
ax2.set_title('Quantum State Amplitudes')

plt.tight_layout()
plt.show()

# Verify encoding
print(f"\nEncoding verification:")
print(f"Input sum of squares: {np.sum(amplitudes**2):.6f}")
print(f"Output sum of squares: {np.sum(state_amplitudes**2):.6f}")
print(f"Maximum difference: {np.max(np.abs(amplitudes - state_amplitudes)):.10f}")

### Amplitude Encoding for EO Image Patches

In [ ]:
def encode_image_patch(img, patch_size=(4, 4)):
    """
    Encode an image patch using amplitude encoding
    
    Args:
        img: Input image
        patch_size: Size of patch to extract
    
    Returns:
        Quantum state representing the patch
    """
    # Extract patch from center
    h, w = img.shape[:2]
    ph, pw = patch_size
    
    if len(img.shape) == 3:
        img_gray = np.mean(img, axis=2)
    else:
        img_gray = img
    
    # Extract center patch
    start_h, start_w = (h - ph) // 2, (w - pw) // 2
    patch = img_gray[start_h:start_h+ph, start_w:start_w+pw]
    
    # Flatten and normalize
    patch_flat = patch.flatten() / 255.0
    patch_norm = patch_flat / (np.linalg.norm(patch_flat) + 1e-10)
    
    return patch, patch_norm

# Encode patches from different land cover types
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

for idx, (class_name, img) in enumerate(samples[:3]):
    # Extract and encode patch
    patch, encoded = encode_image_patch(img, patch_size=(4, 4))
    n_qubits_patch = int(np.ceil(np.log2(len(encoded))))
    
    # Pad to power of 2
    encoded_padded = np.zeros(2**n_qubits_patch)
    encoded_padded[:len(encoded)] = encoded
    encoded_padded = encoded_padded / np.linalg.norm(encoded_padded)
    
    # Show original patch
    axes[0, idx].imshow(patch, cmap='gray')
    axes[0, idx].set_title(f'{class_name}\n4×4 Patch')
    axes[0, idx].axis('off')
    
    # Show encoded amplitudes
    axes[1, idx].bar(range(len(encoded_padded)), encoded_padded, color='green', alpha=0.7)
    axes[1, idx].set_title(f'Quantum Amplitudes\n({n_qubits_patch} qubits)')
    axes[1, idx].set_xlabel('Basis State')
    axes[1, idx].set_ylabel('Amplitude')

plt.tight_layout()
plt.show()

# Part 3: Angle Encoding (Phase Encoding)

## Theory

**Angle encoding** (also called phase encoding) encodes classical data into the rotation angles of quantum gates, creating superposition states.

### Mathematical Formulation

For a classical vector $\mathbf{x} = (x_1, x_2, ..., x_n)$, angle encoding applies rotations:

$$|\psi\rangle = \bigotimes_{i=1}^{n} R(\theta_i) |0\rangle$$

where $\theta_i = f(x_i)$ and $R$ is typically $R_X$, $R_Y$, or $R_Z$:

$$R_Y(\theta) = \begin{pmatrix} \cos(\theta/2) & -\sin(\theta/2) \\ \sin(\theta/2) & \cos(\theta/2) \end{pmatrix}$$

### Properties:
- **Linear capacity**: $n$ qubits encode $n$ classical values
- **Natural superposition**: Creates entangled states
- **Scaling flexibility**: Can use $\theta = x$, $\theta = \pi x$, etc.
- **Use case**: Good for continuous features with physical meaning

### For Earth Observation:
Ideal for encoding:
- Spectral band intensities (each band → rotation angle)
- Vegetation indices (NDVI, EVI)
- Temporal features (seasonal patterns)

In [ ]:
def angle_encoding_prep(features, scaling='linear'):
    """
    Prepare angles for angle encoding
    
    Args:
        features: Feature vector [0, 1]
        scaling: 'linear' (θ=πx) or 'arcsin' (θ=2arcsin(√x))
    
    Returns:
        Angle vector
    """
    if scaling == 'linear':
        # Scale to [0, π]
        angles = np.pi * features
    elif scaling == 'arcsin':
        # Arcsin scaling for amplitude-like encoding
        angles = 2 * np.arcsin(np.sqrt(features))
    else:
        angles = features
    
    return angles

# Prepare angles
angles = angle_encoding_prep(features[:n_qubits], scaling='linear')

print("Angle Encoding Preparation:")
print(f"Original features: {features[:n_qubits]}")
print(f"Rotation angles:   {angles}")
print(f"Angles (degrees):  {np.degrees(angles)}")

### Implementing Angle Encoding in PennyLane

In [ ]:
dev_angle = qml.device('default.qubit', wires=n_qubits)

@qml.qnode(dev_angle)
def angle_encoding_circuit(angles, rotation_gate='RY'):
    """
    Create quantum circuit with angle encoding
    
    Args:
        angles: Rotation angles for each qubit
        rotation_gate: 'RX', 'RY', or 'RZ'
    """
    # Apply rotation gates
    for i, angle in enumerate(angles):
        if rotation_gate == 'RX':
            qml.RX(angle, wires=i)
        elif rotation_gate == 'RY':
            qml.RY(angle, wires=i)
        elif rotation_gate == 'RZ':
            qml.RZ(angle, wires=i)
    
    return qml.state()

# Compare different rotation gates
rotation_types = ['RX', 'RY', 'RZ']
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

for idx, rot_type in enumerate(rotation_types):
    quantum_state = angle_encoding_circuit(angles, rotation_gate=rot_type)
    probabilities = np.abs(quantum_state) ** 2
    
    # Plot probabilities
    axes[idx].bar(range(len(probabilities)), probabilities, color='purple', alpha=0.7)
    axes[idx].set_xlabel('Basis State')
    axes[idx].set_ylabel('Probability')
    axes[idx].set_title(f'Angle Encoding with {rot_type} Gates')
    axes[idx].set_xlim(-1, 2**n_qubits)

plt.tight_layout()
plt.show()

### Visualizing Angle Encoding on Bloch Sphere

For a single qubit, we can visualize how angle encoding places the qubit state on the Bloch sphere.

In [ ]:
def single_qubit_angle_encoding(angle, rotation='RY'):
    """Encode single feature with angle encoding"""
    dev = qml.device('default.qubit', wires=1)
    
    @qml.qnode(dev)
    def circuit():
        if rotation == 'RX':
            qml.RX(angle, wires=0)
        elif rotation == 'RY':
            qml.RY(angle, wires=0)
        elif rotation == 'RZ':
            qml.RZ(angle, wires=0)
        return qml.state()
    
    return circuit()

# Test different feature values
feature_values = np.linspace(0, 1, 5)
angles_test = np.pi * feature_values

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Show feature to angle mapping
ax1.plot(feature_values, angles_test, 'o-', linewidth=2, markersize=8)
ax1.set_xlabel('Feature Value')
ax1.set_ylabel('Rotation Angle (radians)')
ax1.set_title('Feature → Angle Mapping')
ax1.grid(True, alpha=0.3)
ax1.axhline(y=np.pi, color='r', linestyle='--', alpha=0.5, label='π')
ax1.legend()

# Show resulting state amplitudes
for feature, angle in zip(feature_values, angles_test):
    state = single_qubit_angle_encoding(angle, 'RY')
    prob_0 = np.abs(state[0])**2
    prob_1 = np.abs(state[1])**2
    
    ax2.bar([feature - 0.02, feature + 0.02], [prob_0, prob_1], width=0.03, 
            label=f'x={feature:.2f}' if feature in [0, 0.5, 1.0] else '')

ax2.set_xlabel('Feature Value')
ax2.set_ylabel('Measurement Probability')
ax2.set_title('Quantum State Probabilities: P(|0⟩) vs P(|1⟩)')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### Angle Encoding for Spectral Band Analysis

In [ ]:
def extract_spectral_features(img, n_bands=3):
    """
    Extract mean spectral values from image
    
    Args:
        img: RGB image
        n_bands: Number of spectral bands
    
    Returns:
        Spectral feature vector
    """
    if len(img.shape) == 3:
        # Mean value per channel
        spectral_features = [np.mean(img[:, :, i]) / 255.0 for i in range(min(n_bands, img.shape[2]))]
    else:
        spectral_features = [np.mean(img) / 255.0]
    
    return np.array(spectral_features)

# Compare spectral signatures
fig, axes = plt.subplots(2, len(samples), figsize=(15, 6))

spectral_data = []
for idx, (class_name, img) in enumerate(samples):
    # Show image
    axes[0, idx].imshow(img)
    axes[0, idx].set_title(class_name)
    axes[0, idx].axis('off')
    
    # Extract and encode spectral features
    spectral = extract_spectral_features(img, n_bands=3)
    spectral_data.append((class_name, spectral))
    angles_spectral = angle_encoding_prep(spectral, scaling='linear')
    
    # Encode in quantum circuit
    dev_spectral = qml.device('default.qubit', wires=3)
    
    @qml.qnode(dev_spectral)
    def spectral_circuit(angles):
        for i, angle in enumerate(angles):
            qml.RY(angle, wires=i)
        return qml.probs(wires=[0, 1, 2])
    
    probs = spectral_circuit(angles_spectral)
    
    # Plot probability distribution
    axes[1, idx].bar(range(8), probs, color='teal', alpha=0.7)
    axes[1, idx].set_xlabel('Basis State')
    axes[1, idx].set_ylabel('Probability')
    axes[1, idx].set_title(f'R={spectral[0]:.2f}, G={spectral[1]:.2f}, B={spectral[2]:.2f}')

plt.tight_layout()
plt.show()

# Part 4: Hamiltonian Encoding (Time Evolution)

## Theory

**Hamiltonian encoding** embeds data by using it to parameterize a Hamiltonian and then evolving a quantum state under that Hamiltonian.

### Mathematical Formulation

Given classical data $\mathbf{x}$, we construct a Hamiltonian:

$$H(\mathbf{x}) = \sum_{i} x_i H_i$$

where $H_i$ are Pauli operators. The quantum state is then:

$$|\psi(\mathbf{x})\rangle = e^{-iH(\mathbf{x})t} |0\rangle^{\otimes n}$$

Common choices:
- **Single-qubit**: $H = x \sigma_Z$ → $e^{-ix\sigma_Z t} = R_Z(2xt)$
- **Two-qubit**: $H = x (\sigma_Z \otimes \sigma_Z)$ → Creates entanglement

### Properties:
- **Physical interpretation**: Mimics real quantum evolution
- **Entanglement**: Naturally creates correlations between qubits
- **Time parameter**: Additional tuning parameter $t$
- **Use case**: When data represents physical quantities or correlations

### For Earth Observation:
Useful for:
- Encoding correlations between spectral bands
- Temporal evolution of land cover
- Multi-sensor data fusion (correlations between sensors)

In [ ]:
def hamiltonian_encoding_prep(features, interaction_strength=1.0):
    """
    Prepare parameters for Hamiltonian encoding
    
    Args:
        features: Feature vector
        interaction_strength: Scaling factor for evolution time
    
    Returns:
        Evolution parameters
    """
    # Scale features to appropriate range
    # For ZZ interactions, features should be scaled
    evolution_times = interaction_strength * features
    
    return evolution_times

# Prepare evolution parameters
evolution_params = hamiltonian_encoding_prep(features[:n_qubits], interaction_strength=np.pi)

print("Hamiltonian Encoding Parameters:")
print(f"Features:        {features[:n_qubits]}")
print(f"Evolution times: {evolution_params}")

### Implementing Hamiltonian Encoding in PennyLane

In [ ]:
dev_hamiltonian = qml.device('default.qubit', wires=n_qubits)

@qml.qnode(dev_hamiltonian)
def hamiltonian_encoding_circuit(evolution_times, encoding_type='single'):
    """
    Create quantum circuit with Hamiltonian encoding
    
    Args:
        evolution_times: Evolution time parameters
        encoding_type: 'single' (non-interacting) or 'pairwise' (interacting)
    """
    n = len(evolution_times)
    
    # Initialize in superposition
    for i in range(n):
        qml.Hadamard(wires=i)
    
    if encoding_type == 'single':
        # Single-qubit evolutions: exp(-i x_i Z_i t)
        for i, t in enumerate(evolution_times):
            qml.RZ(2 * t, wires=i)
    
    elif encoding_type == 'pairwise':
        # Pairwise interactions: exp(-i x_i Z_i ⊗ Z_{i+1} t)
        for i in range(n - 1):
            t = evolution_times[i]
            # ZZ interaction = CNOT + RZ + CNOT
            qml.CNOT(wires=[i, i+1])
            qml.RZ(2 * t, wires=i+1)
            qml.CNOT(wires=[i, i+1])
    
    return qml.state()

# Compare single vs pairwise encoding
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

for idx, enc_type in enumerate(['single', 'pairwise']):
    quantum_state = hamiltonian_encoding_circuit(evolution_params, encoding_type=enc_type)
    probabilities = np.abs(quantum_state) ** 2
    
    # Plot probabilities
    axes[idx].bar(range(len(probabilities)), probabilities, color='orange', alpha=0.7)
    axes[idx].set_xlabel('Basis State')
    axes[idx].set_ylabel('Probability')
    axes[idx].set_title(f'Hamiltonian Encoding ({enc_type.capitalize()})')
    axes[idx].set_xlim(-1, min(64, 2**n_qubits))

plt.tight_layout()
plt.show()

# Show entanglement difference
state_single = hamiltonian_encoding_circuit(evolution_params, encoding_type='single')
state_pairwise = hamiltonian_encoding_circuit(evolution_params, encoding_type='pairwise')

print("\nComparison of Encoding Types:")
print(f"Single-qubit encoding: Max probability = {np.max(np.abs(state_single)**2):.4f}")
print(f"Pairwise encoding:     Max probability = {np.max(np.abs(state_pairwise)**2):.4f}")
print(f"Number of non-zero amplitudes (single):   {np.sum(np.abs(state_single)**2 > 1e-6)}")
print(f"Number of non-zero amplitudes (pairwise): {np.sum(np.abs(state_pairwise)**2 > 1e-6)}")

### Time Evolution Encoding: Temporal Analysis

For EO applications, we can use time evolution to encode temporal changes in land cover.

In [ ]:
def time_evolution_encoding(feature, time_steps=10):
    """
    Show evolution of quantum state over time
    
    Args:
        feature: Single feature value
        time_steps: Number of time steps
    
    Returns:
        Evolution trajectory
    """
    dev = qml.device('default.qubit', wires=1)
    times = np.linspace(0, 2*np.pi, time_steps)
    states = []
    
    for t in times:
        @qml.qnode(dev)
        def evolve():
            qml.Hadamard(wires=0)
            qml.RZ(2 * feature * t, wires=0)
            return qml.state()
        
        states.append(evolve())
    
    return times, np.array(states)

# Visualize time evolution
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

feature_vals = [0.2, 0.5, 0.8]
for idx, feat in enumerate(feature_vals):
    times, states = time_evolution_encoding(feat, time_steps=50)
    
    # Calculate probabilities
    prob_0 = np.abs(states[:, 0])**2
    prob_1 = np.abs(states[:, 1])**2
    
    axes[idx].plot(times, prob_0, label='P(|0⟩)', linewidth=2)
    axes[idx].plot(times, prob_1, label='P(|1⟩)', linewidth=2)
    axes[idx].set_xlabel('Evolution Time')
    axes[idx].set_ylabel('Probability')
    axes[idx].set_title(f'Time Evolution (feature = {feat})')
    axes[idx].legend()
    axes[idx].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### Hamiltonian Encoding for Multi-Band Correlation

In [ ]:
def encode_spectral_correlation(img1, img2):
    """
    Encode correlation between two spectral signatures using Hamiltonian
    
    Args:
        img1, img2: Two images to compare
    
    Returns:
        Quantum states representing the correlation
    """
    # Extract spectral features
    spec1 = extract_spectral_features(img1, n_bands=3)
    spec2 = extract_spectral_features(img2, n_bands=3)
    
    # Compute correlation-based features
    correlation = spec1 * spec2  # Element-wise product
    
    return spec1, spec2, correlation

# Compare different land cover types
img_forest = samples[1][1]  # Forest
img_urban = samples[3][1]   # Urban (Highway or similar)

spec_forest, spec_urban, corr_features = encode_spectral_correlation(img_forest, img_urban)

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Plot spectral signatures
x = ['Red', 'Green', 'Blue']
axes[0].bar(x, spec_forest, alpha=0.7, label='Forest', color='green')
axes[0].bar(x, spec_urban, alpha=0.7, label='Urban', color='gray')
axes[0].set_ylabel('Normalized Intensity')
axes[0].set_title('Spectral Signatures')
axes[0].legend()

# Plot correlation features
axes[1].bar(x, corr_features, color='purple', alpha=0.7)
axes[1].set_ylabel('Correlation Value')
axes[1].set_title('Spectral Correlation Features')

# Encode using Hamiltonian and show quantum state
dev_corr = qml.device('default.qubit', wires=3)

@qml.qnode(dev_corr)
def correlation_circuit(corr_features):
    # Initialize in superposition
    for i in range(3):
        qml.Hadamard(wires=i)
    
    # Apply pairwise interactions based on correlation
    for i in range(2):
        qml.CNOT(wires=[i, i+1])
        qml.RZ(2 * np.pi * corr_features[i], wires=i+1)
        qml.CNOT(wires=[i, i+1])
    
    return qml.probs(wires=[0, 1, 2])

probs = correlation_circuit(corr_features)
axes[2].bar(range(8), probs, color='teal', alpha=0.7)
axes[2].set_xlabel('Basis State')
axes[2].set_ylabel('Probability')
axes[2].set_title('Encoded Quantum State')

plt.tight_layout()
plt.show()

# Part 5: Phase Encoding (IQP-Style Encoding)

## Theory

**Phase encoding** (distinct from angle encoding) encodes data into the relative phases of quantum states, often using diagonal operators and creating highly entangled states.

### Mathematical Formulation

Starting from an equal superposition, apply phase rotations:

$|\psi\rangle = \frac{1}{\sqrt{2^n}} \sum_{i=0}^{2^n-1} e^{i\phi(i, \mathbf{x})} |i\rangle$

where $\phi(i, \mathbf{x})$ depends on both the basis state $i$ and the data $\mathbf{x}$.

**IQP-style encoding**: Uses diagonal unitaries in the Hadamard basis:

1. Apply Hadamard to all qubits: $H^{\otimes n}|0\rangle^{\otimes n}$
2. Apply data-dependent phase: $e^{i\sum_{j,k} x_{jk} Z_j Z_k}$
3. Apply Hadamard again: $H^{\otimes n}$

### Properties:
- **Phase-based**: Information in relative phases, not amplitudes
- **Computational hardness**: IQP circuits are classically hard to simulate
- **Interference**: Exploits quantum interference effects
- **Use case**: Quantum advantage in classification tasks

### For Earth Observation:
Applications include:
- Feature interactions in classification
- Non-linear transformations of spectral data
- Exploiting quantum interference for pattern recognition

In [ ]:
def phase_encoding_prep(features):
    """
    Prepare phase parameters for encoding
    
    Args:
        features: Feature vector
    
    Returns:
        Phase parameters
    """
    # Scale features to appropriate phase range
    phases = 2 * np.pi * features
    return phases

# Prepare phases
phases = phase_encoding_prep(features[:n_qubits])

print("Phase Encoding Parameters:")
print(f"Features: {features[:n_qubits]}")
print(f"Phases:   {phases}")
print(f"Phases (degrees): {np.degrees(phases)}")

### Implementing Phase Encoding in PennyLane

In [ ]:
dev_phase = qml.device('default.qubit', wires=n_qubits)

@qml.qnode(dev_phase)
def phase_encoding_circuit(phases, encoding_type='simple'):
    """
    Create quantum circuit with phase encoding
    
    Args:
        phases: Phase parameters for each qubit
        encoding_type: 'simple' (single-qubit) or 'iqp' (IQP-style)
    """
    n = len(phases)
    
    # Create equal superposition
    for i in range(n):
        qml.Hadamard(wires=i)
    
    if encoding_type == 'simple':
        # Simple phase encoding: apply phase to each qubit
        for i, phase in enumerate(phases):
            qml.PhaseShift(phase, wires=i)
    
    elif encoding_type == 'iqp':
        # IQP-style encoding with ZZ interactions
        for i in range(n):
            qml.RZ(phases[i], wires=i)
        
        # Add pairwise ZZ interactions
        for i in range(n - 1):
            qml.CNOT(wires=[i, i+1])
            qml.RZ(phases[i] * phases[i+1], wires=i+1)
            qml.CNOT(wires=[i, i+1])
        
        # Final Hadamard layer (IQP style)
        for i in range(n):
            qml.Hadamard(wires=i)
    
    return qml.state()

# Compare encoding types
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Simple phase encoding
state_simple = phase_encoding_circuit(phases, encoding_type='simple')
probs_simple = np.abs(state_simple) ** 2

axes[0, 0].bar(range(len(state_simple)), np.real(state_simple), alpha=0.7, label='Real')
axes[0, 0].bar(range(len(state_simple)), np.imag(state_simple), alpha=0.7, label='Imag')
axes[0, 0].set_xlabel('Basis State')
axes[0, 0].set_ylabel('Amplitude')
axes[0, 0].set_title('Simple Phase Encoding - Amplitudes')
axes[0, 0].legend()
axes[0, 0].set_xlim(-1, min(64, 2**n_qubits))

axes[0, 1].bar(range(len(probs_simple)), probs_simple, color='blue', alpha=0.7)
axes[0, 1].set_xlabel('Basis State')
axes[0, 1].set_ylabel('Probability')
axes[0, 1].set_title('Simple Phase Encoding - Probabilities')
axes[0, 1].set_xlim(-1, min(64, 2**n_qubits))

# IQP-style phase encoding
state_iqp = phase_encoding_circuit(phases, encoding_type='iqp')
probs_iqp = np.abs(state_iqp) ** 2

axes[1, 0].bar(range(len(state_iqp)), np.real(state_iqp), alpha=0.7, label='Real')
axes[1, 0].bar(range(len(state_iqp)), np.imag(state_iqp), alpha=0.7, label='Imag')
axes[1, 0].set_xlabel('Basis State')
axes[1, 0].set_ylabel('Amplitude')
axes[1, 0].set_title('IQP-Style Phase Encoding - Amplitudes')
axes[1, 0].legend()
axes[1, 0].set_xlim(-1, min(64, 2**n_qubits))

axes[1, 1].bar(range(len(probs_iqp)), probs_iqp, color='red', alpha=0.7)
axes[1, 1].set_xlabel('Basis State')
axes[1, 1].set_ylabel('Probability')
axes[1, 1].set_title('IQP-Style Phase Encoding - Probabilities')
axes[1, 1].set_xlim(-1, min(64, 2**n_qubits))

plt.tight_layout()
plt.show()

print("\nPhase Encoding Comparison:")
print(f"Simple encoding - Entropy: {-np.sum(probs_simple * np.log2(probs_simple + 1e-10)):.4f}")
print(f"IQP encoding - Entropy:    {-np.sum(probs_iqp * np.log2(probs_iqp + 1e-10)):.4f}")

### Phase Encoding for Feature Interactions

Phase encoding is particularly useful for capturing interactions between features.

In [ ]:
def visualize_feature_interactions(features_list, labels):
    """
    Visualize how different feature sets are encoded with phase encoding
    
    Args:
        features_list: List of feature vectors
        labels: Labels for each feature vector
    """
    n_features = len(features_list[0])
    dev = qml.device('default.qubit', wires=n_features)
    
    fig, axes = plt.subplots(len(features_list), 2, figsize=(14, 4*len(features_list)))
    
    for idx, (features, label) in enumerate(zip(features_list, labels)):
        phases = phase_encoding_prep(features)
        
        @qml.qnode(dev)
        def encode_features():
            # IQP-style encoding
            for i in range(n_features):
                qml.Hadamard(wires=i)
            
            for i in range(n_features):
                qml.RZ(phases[i], wires=i)
            
            for i in range(n_features - 1):
                qml.CNOT(wires=[i, i+1])
                qml.RZ(phases[i] * phases[i+1], wires=i+1)
                qml.CNOT(wires=[i, i+1])
            
            for i in range(n_features):
                qml.Hadamard(wires=i)
            
            return qml.state()
        
        state = encode_features()
        probs = np.abs(state) ** 2
        
        # Plot features
        axes[idx, 0].bar(range(n_features), features, color='blue', alpha=0.7)
        axes[idx, 0].set_xlabel('Feature Index')
        axes[idx, 0].set_ylabel('Feature Value')
        axes[idx, 0].set_title(f'{label} - Input Features')
        axes[idx, 0].set_ylim(0, 1)
        
        # Plot quantum state probabilities
        axes[idx, 1].bar(range(len(probs)), probs, color='green', alpha=0.7)
        axes[idx, 1].set_xlabel('Basis State')
        axes[idx, 1].set_ylabel('Probability')
        axes[idx, 1].set_title(f'{label} - Encoded Quantum State')
        axes[idx, 1].set_xlim(-1, min(64, 2**n_features))
    
    plt.tight_layout()
    plt.show()

# Extract features from different land cover types
feature_vectors = []
feature_labels = []

for class_name, img in samples[:3]:
    feat = preprocess_image(img, target_size=8)
    feature_vectors.append(feat)
    feature_labels.append(class_name)

visualize_feature_interactions(feature_vectors, feature_labels)

# Part 6: Comparative Analysis of Encoding Methods

Now let's compare all five encoding methods side-by-side using the same EO data.

In [ ]:
def encode_with_all_methods(features, n_qubits=4):
    """
    Encode features using all five methods
    
    Args:
        features: Feature vector
        n_qubits: Number of qubits to use
    
    Returns:
        Dictionary of encoded states
    """
    features_subset = features[:n_qubits]
    dev = qml.device('default.qubit', wires=n_qubits)
    results = {}
    
    # 1. Basis Encoding
    @qml.qnode(dev)
    def basis_enc():
        binary = basis_encoding(features_subset)
        for i, bit in enumerate(binary):
            if bit == 1:
                qml.PauliX(wires=i)
        return qml.probs(wires=range(n_qubits))
    
    results['Basis'] = basis_enc()
    
    # 2. Amplitude Encoding
    amplitudes_prep, n_q = amplitude_encoding_prep(features_subset)
    if n_q == n_qubits:
        @qml.qnode(dev)
        def amplitude_enc():
            qml.AmplitudeEmbedding(features=amplitudes_prep, wires=range(n_qubits), normalize=True)
            return qml.probs(wires=range(n_qubits))
        
        results['Amplitude'] = amplitude_enc()
    
    # 3. Angle Encoding
    @qml.qnode(dev)
    def angle_enc():
        angles = angle_encoding_prep(features_subset)
        for i, angle in enumerate(angles):
            qml.RY(angle, wires=i)
        return qml.probs(wires=range(n_qubits))
    
    results['Angle'] = angle_enc()
    
    # 4. Hamiltonian Encoding
    @qml.qnode(dev)
    def hamiltonian_enc():
        for i in range(n_qubits):
            qml.Hadamard(wires=i)
        
        evolution_times = hamiltonian_encoding_prep(features_subset)
        for i in range(n_qubits - 1):
            qml.CNOT(wires=[i, i+1])
            qml.RZ(2 * evolution_times[i], wires=i+1)
            qml.CNOT(wires=[i, i+1])
        
        return qml.probs(wires=range(n_qubits))
    
    results['Hamiltonian'] = hamiltonian_enc()
    
    # 5. Phase Encoding
    @qml.qnode(dev)
    def phase_enc():
        phases = phase_encoding_prep(features_subset)
        
        for i in range(n_qubits):
            qml.Hadamard(wires=i)
        
        for i in range(n_qubits):
            qml.RZ(phases[i], wires=i)
        
        for i in range(n_qubits - 1):
            qml.CNOT(wires=[i, i+1])
            qml.RZ(phases[i] * phases[i+1], wires=i+1)
            qml.CNOT(wires=[i, i+1])
        
        for i in range(n_qubits):
            qml.Hadamard(wires=i)
        
        return qml.probs(wires=range(n_qubits))
    
    results['Phase (IQP)'] = phase_enc()
    
    return results

# Compare all methods
fig, axes = plt.subplots(2, 3, figsize=(16, 10))
axes = axes.flatten()

# Original features
sample_features = preprocess_image(samples[0][1], target_size=16)
encoded_states = encode_with_all_methods(sample_features, n_qubits=4)

# Plot original features
axes[0].bar(range(4), sample_features[:4], color='gray', alpha=0.7)
axes[0].set_xlabel('Feature Index')
axes[0].set_ylabel('Value')
axes[0].set_title('Original Features')
axes[0].set_ylim(0, 1)

# Plot each encoding
for idx, (method_name, probs) in enumerate(encoded_states.items(), 1):
    axes[idx].bar(range(len(probs)), probs, alpha=0.7)
    axes[idx].set_xlabel('Basis State')
    axes[idx].set_ylabel('Probability')
    axes[idx].set_title(f'{method_name} Encoding')
    axes[idx].set_xlim(-0.5, min(16, len(probs)-0.5))

plt.tight_layout()
plt.show()

### Quantitative Comparison of Encoding Methods

In [ ]:
def analyze_encoding_properties(encoded_states):
    """
    Analyze properties of different encoding methods
    
    Args:
        encoded_states: Dictionary of probability distributions
    
    Returns:
        Analysis results
    """
    properties = {}
    
    for method, probs in encoded_states.items():
        # Shannon entropy (measure of distribution spread)
        entropy = -np.sum(probs * np.log2(probs + 1e-10))
        
        # Effective dimension (inverse participation ratio)
        eff_dim = 1.0 / np.sum(probs ** 2)
        
        # Maximum probability
        max_prob = np.max(probs)
        
        # Number of significant basis states (prob > 1%)
        n_significant = np.sum(probs > 0.01)
        
        properties[method] = {
            'Entropy': entropy,
            'Effective Dimension': eff_dim,
            'Max Probability': max_prob,
            'Significant States': n_significant
        }
    
    return properties

# Analyze encoding properties
properties = analyze_encoding_properties(encoded_states)

# Create comparison table
print("\n" + "="*80)
print("COMPARATIVE ANALYSIS OF QUANTUM ENCODING METHODS")
print("="*80)
print(f"{'Method':<20} {'Entropy':<12} {'Eff. Dim':<12} {'Max Prob':<12} {'Sig. States':<12}")
print("-"*80)

for method, props in properties.items():
    print(f"{method:<20} {props['Entropy']:<12.4f} {props['Effective Dimension']:<12.4f} "
          f"{props['Max Probability']:<12.4f} {props['Significant States']:<12}")

print("="*80)

# Visualize comparison
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

metrics = ['Entropy', 'Effective Dimension', 'Max Probability', 'Significant States']
for idx, metric in enumerate(metrics):
    ax = axes[idx // 2, idx % 2]
    values = [props[metric] for props in properties.values()]
    methods = list(properties.keys())
    
    colors = ['blue', 'green', 'orange', 'purple', 'red']
    bars = ax.bar(range(len(methods)), values, color=colors, alpha=0.7)
    ax.set_xticks(range(len(methods)))
    ax.set_xticklabels(methods, rotation=45, ha='right')
    ax.set_ylabel(metric)
    ax.set_title(f'Comparison: {metric}')
    ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

### Interpretation of Results

**Entropy & Effective Dimension:**
- Higher values indicate more spread-out probability distributions
- **Basis encoding**: Lowest entropy (deterministic single state)
- **Amplitude & Angle**: Moderate spread
- **Hamiltonian & Phase**: Highest entropy (many states have significant probability)

**Maximum Probability:**
- **Basis**: Always 1.0 (deterministic)
- **Others**: Lower values indicate superposition

**Significant States:**
- Number of basis states that could be measured with reasonable probability
- Important for quantum advantage in classification

# Summary and Best Practices

## Encoding Method Selection Guide for Earth Observation

| Encoding Method | Best For | Advantages | Disadvantages |
|----------------|----------|------------|---------------|
| **Basis** | Categorical data, binary masks | Simple, deterministic | No superposition, limited expressivity |
| **Amplitude** | High-dim pixel data, spectral signatures | Exponential capacity | Requires normalization, difficult to prepare |
| **Angle** | Continuous features, spectral indices | Natural superposition, easy to implement | Linear capacity |
| **Hamiltonian** | Temporal data, correlations | Physical interpretation, entanglement | More complex, requires careful design |
| **Phase (IQP)** | Classification tasks, feature interactions | Quantum advantage potential | Hard to interpret, requires interference |

# Key Takeaways

1. **Data Preprocessing is Critical**
   - Normalization affects quantum state quality
   - Feature scaling impacts encoding fidelity
   - Dimension reduction may be necessary for amplitude encoding

2. **Trade-offs Between Methods**
   - Expressivity vs. Circuit Depth
   - Classical preprocessing vs. Quantum advantage
   - Interpretability vs. Performance

3. **EO-Specific Considerations**
   - Spectral bands → Angle or Amplitude encoding
   - Spatial features → Hamiltonian encoding for correlations
   - Temporal data → Time evolution encoding
   - Classification tasks → Phase encoding for quantum advantage

4. **Hybrid Approaches**
   - Combine multiple encoding strategies
   - Use classical preprocessing + quantum encoding
   - Layer different encoding methods for hierarchical features

# Conclusion and Further Reading

## What We've Learned

In this notebook, we explored five fundamental quantum encoding methods for Earth observation data:

1. **Basis Encoding** - Direct binary mapping
2. **Amplitude Encoding** - Exponential data compression
3. **Angle Encoding** - Natural superposition states
4. **Hamiltonian Encoding** - Physical time evolution
5. **Phase Encoding** - Quantum interference effects

Each method has unique properties that make it suitable for different types of EO data and machine learning tasks.

## Further Reading

- PennyLane Documentation: https://pennylane.ai/
- Quantum Machine Learning: [arXiv:2101.11020](https://arxiv.org/abs/2101.11020)
- EuroSAT Dataset: [IEEE DataPort](https://ieeexplore.ieee.org/document/8736785)
- Quantum Embeddings: [arXiv:2001.03622](https://arxiv.org/abs/2001.03622)